In [4]:
import glob
import os
from common.lora_resize import change_lora_rank
from tqdm import tqdm
import torch


path = "common/lora_outputs"
files = sorted(glob.glob(f"{path}/**/lora_layers.pth", recursive=True))
new_files = sorted(glob.glob(f"{path}/**/lora_layers_new.pth", recursive=True))

In [7]:
path1 = 'common/lora_outputs/lora-Anne Hathaway/lora_layers.pth'
path2 = 'common/lora_outputs/lora-Anne Hathaway/lora_layers_new.pth'

state_dict1 = torch.load(path1)
state_dict2 = torch.load(path2)

In [17]:
down1 = state_dict1['unet']['down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_out.0.lora_down.weight']
up1 = state_dict1['unet']['down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_out.0.lora_up.weight']

down2 = state_dict2['down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_out.0.lora_down.weight']
up2 = state_dict2['down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_out.0.lora_up.weight']


In [29]:
rand_vect = torch.randn(1,320).cuda()
out1 = (rand_vect @ down1.T) @ up1.T
out2 = (rand_vect.cpu() @ down2.T) @ up2.T

In [30]:
diff = out1.cpu() - out2.cpu()

In [31]:
diff

tensor([[-1.1409e-08, -1.9558e-08,  5.0291e-08, -4.4703e-08, -3.8184e-08,
         -1.7695e-08,  6.5193e-09, -1.3970e-08,  2.6077e-08,  3.8766e-08,
          1.7695e-08, -1.3039e-08, -4.7730e-08, -2.7008e-08,  6.5891e-08,
         -2.9337e-08,  1.1642e-08, -7.9162e-08, -2.7940e-08, -7.4506e-08,
          2.3283e-08, -1.2107e-08,  7.8930e-08,  1.8626e-09, -2.2817e-08,
         -4.0978e-08, -3.0268e-08,  1.6764e-08,  6.5193e-09,  2.5146e-08,
         -3.4459e-08, -1.6764e-08,  1.5832e-08, -1.9092e-08,  2.3516e-08,
          4.7497e-08,  7.9803e-08, -2.5146e-08,  3.8184e-08,  1.0012e-08,
         -1.1525e-08, -3.7486e-08, -1.8626e-08,  5.8208e-09, -2.7008e-08,
          1.7462e-08, -4.7497e-08, -1.7695e-08,  9.8604e-08,  7.2177e-08,
         -9.4064e-08, -3.0734e-08,  4.6566e-09, -2.4913e-08,  3.0966e-08,
         -5.3318e-08, -9.3132e-09, -4.3772e-08,  5.4017e-08,  1.4901e-08,
         -1.2107e-08,  5.1223e-08, -4.6566e-10, -5.3085e-08, -2.1420e-08,
          3.4925e-09, -1.0710e-08,  2.

In [7]:
import torch

path = "/home/ubuntu/AutoLoRADiscovery/lora_bundle.pt"

state_dict = torch.load(path)

In [8]:
from tqdm import tqdm

for i in tqdm(range(len(state_dict))):
    one = state_dict[i]

    for k in list(one.keys()):
        if "lora_down" in k or "lora_up" in k:
            one[k] = one[k].T

    state_dict[i] = one
    

100%|██████████| 136/136 [00:00<00:00, 1758.83it/s]


In [9]:
torch.save(state_dict, path)